# Variational quantum states as partial fourier series

#### [Sashwat Anagolum](https://github.com/SashwatAnagolum)
Mentored by [**Yunong Shi**](https://godott.github.io/) as part of the QOSF mentorship program, spring '21

## Introduction

This notebook was developed as a (tiny) part of the research that Yunong Shi and I worked on over the course of the last two and a half months, and continue to work on, as a part of the QOSF mentorship program. 

\[[1]\] showed that a quantum neural network $f_{\theta}(x)$ is equivalent to partial fourier series in the data $x$:

$$f_{\theta}(x) = \sum_{\omega \: \in \: \Omega} c_{\omega} (\theta) e^{i \omega x}$$

Where $\Omega$ is the set of frequencies accessible to the model, and $c_{\omega}$ are complex-valued weights determining the contribution of each frequency to the model function. 

**Some more intro stuff**

### Required imports

[1]: https://arxiv.org/abs/2008.08605
[2]: https://arxiv.org/abs/2106.03747
[3]: https://arxiv.org/abs/2011.01938
[4]: https://arxiv.org/abs/2101.11020

In [285]:
import numpy as np
from scipy import linalg

from functools import reduce

import pennylane as qml

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D

### Circuit layers

These functions define the different layers we can use in the circuits we construct our kernels from:

In [328]:
def ry_layer(_, params):
    """
    Append a layer of RY gates to the quantum circuit.
    """
    for i, theta in enumerate(params):
        qml.RY(-1 * theta, wires=i)
    

def rx_layer(_, params):
    """
    Append a layer of RX gates to the quantum circuit.
    """
    for i, theta in enumerate(params):
        qml.RX(theta, wires=i)  
        

def rz_layer(_, params):
    """
    Append a layer of RZ gates gates to the quantum circuit.
    """
    for i, theta in enumerate(params):
        qml.RZ(theta, wires=i) 
        

def cr_layer(gate, cr_map, num_qubits, params):
    """
    Append a layer of controlled pauli rotation gates to the quantum circuit.
    """
    gate = {'crz': qml.CRZ, 'cry': qml.CRY, 'crx': qml.CRX}[gate]

    if cr_map == 'linear':
        for i, theta in enumerate(params):
            gate(theta, wires=[i, i + 1])
    elif cr_map == 'full':
        for i in range(num_qubits - 1):
            offset = sum(range(num_qubits - i, num_qubits))
            for j in range(num_qubits - i - 1):
                gate(params[offset + j], wires=[i, i + j + 1])


def hadamard_layer(num_qubits, _):
    """
    Append a layer of hadamard gates to the quantum circuit.
    """
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
        

def c_layer(gate, c_map, num_qubits, _):
    """
    Append a layer of controlled pauli gates to the quantum circuit.
    """
    gate = {'cz': qml.CZ, 'cy': qml.CY, 'cx': qml.CNOT}[gate]
    
    if c_map == 'full':
        for i in range(num_qubits - 1):
            for j in range(num_qubits - i - 1):
                gate(wires=[i, i + j + 1])        
    else:
        for i in range(num_qubits - 1):
            gate(wires=[i, i + 1])

        if c_map == 'circular':
            gate(wires=[num_qubits - 1, 0])        

### Create quantum circuits

In [329]:
def qc_generator(num_qubits, layers, layer_extra_params, weights_bounds, inputs_bounds, ret_type, 
                        state_in, input_transform):
    """
    Generate a variational quantum circuit that accepts data inputs and variational parameters based on the
    selected combination of layers and return type.
    """
    device = qml.device('default.qubit', wires=num_qubits)
    
    @qml.qnode(device)
    def qc(inputs, weights):                
        if state_in:
            qml.QubitStateVector(inputs[1], wires=range(num_qubits))
        
            inputs = inputs[0]
        else:
            inputs = input_transform(inputs)
        
        for i, layer in enumerate(layers):
            data_in = np.concatenate((weights[weights_bounds[i]: weights_bounds[i + 1]],
                                      inputs[inputs_bounds[i]: inputs_bounds[i + 1]]))

            layer(*layer_extra_params[i], data_in)     
        
        if ret_type == 'exp': 
            return [qml.expval(qml.PauliZ(wires=i)) for i in range(2)]
        elif ret_type == 'state':
            return qml.state()
        elif isinstance(ret_type, list) or isinstance(ret_type, tuple):
            return qml.density_matrix(wires=ret_type)
        elif ret_type == 'exp1':
            return qml.expval(qml.PauliZ(wires=0))
    
    return qc

### Utilities

In [325]:
def get_circuit_unitary(layers, layer_params, weights, weight_boundaries, inputs, input_boundaries, num_qubits):
    basis_states = np.eye(2 ** num_qubits)
    circ = qc_generator(num_qubits, layers, layer_params, weight_boundaries, input_boundaries, 
                               'state', True, None)
    unitary = np.stack([circ([inputs, state], weights) for state in basis_states])
    
    return unitary


def get_hamiltonian(layer, layer_extra_params):
    pauli_x = np.array([[0, 1], [1, 0]])
    pauli_y = np.array([[0, 0-1j], [0+1j, 0]])
    pauli_z = np.array([[1, 0], [0, -1]])
    
    if layer == rx_layer:
        h =  pauli_x
    elif layer == ry_layer:
        h = pauli_y
    elif layer == rz_layer:
        h = pauli_z
    else:
        h = layer_extra_params[0]
        
    return h


def print_coeffs(simple_coeffs):
    for i, coeff_set in enumerate(simple_coeffs):
        print('[U(x, θ)|0>]_{} = \n'.format(i))
        for feat in range(len(simple_coeffs[i])):
            combos = list(simple_coeffs[i][feat].keys())
            for j in range(len(combos)):
                print('{} * exp({} * i * x_{}) '.format(simple_coeffs[i][feat][combos[j]], combos[j], feat), end=" ")
                if not (j == len(combos) - 1 and feat == len(simple_coeffs[i]) - 1):
                    print('+')
                else:
                    print()

        print()    

### Frequency computation

In [289]:
def diagonalize_layers(layers, layer_extra_params, num_qubits):
    vs = []
    diag_Hs = []
    
    for i, layer in enumerate(layers):
        hamiltonian = get_hamiltonian(layer, layer_extra_params[i])
        vals, vects = np.linalg.eig(hamiltonian)
        
        v = vects
        
        for i in range(num_qubits - 1):
            v = np.kron(v, vects)

        vs.append(v)
        diag_H = np.zeros((2 ** num_qubits, 2 ** num_qubits)).astype('complex128')
        
        for j in range(num_qubits):
            ops = [np.eye(2) for i in range(j)] + [np.diag(vals)] + [np.eye(2) for i in range(j + 1, num_qubits)]
            diag_H += reduce(lambda a, b: np.kron(a, b), ops)
            
        diag_Hs.append(0.5 * np.diag(diag_H))

    return diag_Hs, vs

            
def compute_freqs(eig_vals, num_qubits_per_feat=1):
    num_embeddings = len(eig_vals)
    num_qubits = int(np.log2(len(eig_vals[0])))
    num_feats = num_qubits // num_qubits_per_feat
    kernel_freqs = []
    pauli_eigs = [0.5, -0.5]  
    
    for i in range((2 ** num_qubits) ** num_embeddings):
        i_b = []
        sub_i_bs = []
        current_num = i
        
        for j in range(num_embeddings):
            i_b.append(current_num // ((2 ** num_qubits) ** (num_embeddings - j - 1)))
            current_num %= ((2 ** num_qubits) ** (num_embeddings - j - 1))
            
        for j in i_b:
            sub_i_b = []
            current_num = j
            
            for k in range(num_feats):
                sub_i_b.append(current_num // (2 ** (num_feats - k - 1)))
                current_num %= (2 ** (num_feats - k - 1))
                
            sub_i_bs.append(sub_i_b)
        
        if num_qubits_per_feat == 1:
            kernel_freqs.append([sum([pauli_eigs[sub_i_bs[j][i]] 
                                            for j in range(num_embeddings)]) for i in range(num_feats)])
        else:
            eigval_sum_i = sum([np.round(eig_vals[idx][i], 1) for idx, i in enumerate(i_b)])
            kernel_freqs.append(eigval_sum_i)   

    return [tuple(freqs) for freqs in kernel_freqs]

### Coefficient computation

In [290]:
def get_embedding_layers(layers, input_bounds):
    indices = []
    embedding_layers = []
    
    for i, layer in enumerate(layers):
        if input_bounds[i + 1] - input_bounds[i]:
            embedding_layers.append(layer)
            indices.append(i)
            
    return embedding_layers, indices


def get_unitaries(layers, layer_params, weights, weight_bounds, input_bounds, num_qubits):
    init_unitaries = []
    indices = []
    trimmed_indices = []
    inputs = [0 for i in range(input_bounds[-1])]
    i = 0
    
    for i, layer in enumerate(layers):
        if input_bounds[i] == input_bounds[i + 1]:
            indices.append(i)
            
    last_break = indices[0]
    
    for i in range(1, len(indices)):
        if indices[i] - indices[i - 1] > 1:
            trimmed_indices.append([last_break, indices[i - 1] + 1])
            last_break = indices[i]
            
    trimmed_indices.append([last_break, indices[-1] + 1])
            
    for i in range(len(trimmed_indices)):
        unitary_layers = layers[trimmed_indices[i][0]: trimmed_indices[i][1]]
        unitary_layer_params = layer_params[trimmed_indices[i][0]: trimmed_indices[i][1]]
        unitary_weight_bounds = weight_bounds[trimmed_indices[i][0]:trimmed_indices[i][1] + 1]
        unitary_input_bounds = input_bounds[trimmed_indices[i][0]:trimmed_indices[i][1] + 1]
        init_unitaries.append(get_circuit_unitary(unitary_layers, unitary_layer_params, weights, 
                                                  unitary_weight_bounds, inputs, unitary_input_bounds, 
                                                  num_qubits))
        
    return init_unitaries, trimmed_indices


def update_unitaries(init_unitaries, v_daggers, unitary_indices):
    if not len(v_daggers):
        return init_unitaries
    
    init_unitaries = [init_unitary for init_unitary in init_unitaries]
    updated_unitaries = [] 
    if unitary_indices[0][0]:
        init_unitaries.insert(0, np.eye(init_unitaries[0].shape[0]))
        
    updated_unitaries.append(np.matmul(init_unitaries[0], v_daggers[0]))
    
    for i in range(1, len(init_unitaries) - 1):
        updated_unitaries.append(np.matmul(np.conj(v_daggers[i - 1].T), np.matmul(init_unitaries[i], v_daggers[i])))
    
    updated_unitaries.append(np.matmul(np.conj(v_daggers[-1].T), init_unitaries[len(v_daggers)]))
    
    return updated_unitaries


def compute_kernel_coeffs(unitaries, eig_vals, kernel_freqs, num_feats):
    coeffs = [{freq:0 for freq in list(set(kernel_freqs))} for i in range(eig_vals[0].shape[0])]
    num_embeddings = len(eig_vals)
    freqs = []
    
    for i in range(eig_vals[0].shape[0]):
        for j in range((2 ** num_qubits) ** num_embeddings):
            j_b = [0]
            current_num = j
            
            for k in range(num_embeddings):
                j_b.append(current_num // ((2 ** num_qubits) ** (num_embeddings - k - 1)))
                current_num %= ((2 ** num_qubits) ** (num_embeddings - k - 1))
             
            j_b.append(i)
            freq_combo = kernel_freqs[j]
            prod = 1
            for idx, unitary in enumerate(unitaries):
                prod *= unitary[j_b[idx], j_b[idx + 1]]

            coeffs[i][freq_combo] += prod

    return coeffs

### Create quantum states from collections of frequencies and coefficients

In [265]:
def get_state_from_coeffs(inputs, coeffs, num_feats):
    state = np.zeros(len(coeffs)).astype('complex128')
    
    for i in range(len(coeffs)):
        for freq_combo, weight in coeffs[i].items():
            state[i] += weight * np.exp(-1j * sum([freq * inputs[idx] for idx, freq in enumerate(list(freq_combo))]))
    
    return state

### Get simplified coefficients

In [306]:
def simplify_coeffs(coeffs):
    num_feats = len(list(coeffs[0].keys())[0])
    simple_coeffs = [[{freq:0 for freq in set([combo[j] for combo in coeffs[i].keys()])} 
                      for j in range(num_feats)] for i in range(len(coeffs))]
    
    for i in range(len(coeffs)):
        for freq_combo in coeffs[i].keys():
            for feat in range(num_feats):
                 simple_coeffs[i][feat][freq_combo[feat]] += coeffs[i][freq_combo]
    
    return simple_coeffs

### Testing the quantum circuit to partial fourier series method

If the conversion works right, then we should get the same state by running a quantum circuit and outputting a statevector and by converting it into a $2^n$-dimensional partial fourier series and evaluating the value of the series at the data input values.

First, we run a small (or big!) quantum circuit with random weights and input values:

In [330]:
num_qubits = 4
layers = [rx_layer, c_layer, rx_layer]
layer_params = [[None], ['cx', 'linear', num_qubits], [None]]
weight_boundaries = [0, 0, 0, 4]
input_boundaries = [0, 4, 4, 4]
weights = 2 * np.pi * np.random.sample(2)
inputs = [*list(np.random.sample(4))] * 1

circ = qc_generator(num_qubits, layers, layer_params, weight_boundaries, input_boundaries, 'state', False, lambda x: x)
state_qc = circ(inputs, weights)
state_qc

tensor([ 0.21962078+0.05389356j,  0.01336267-0.05445399j,
        -0.0049505 +0.01924282j, -0.07760906-0.0199661j ,
        -0.00712376-0.21751854j, -0.05393275+0.0017663j ,
        -0.0194288 +0.03745217j, -0.15105   -0.07835913j,
         0.08427376-0.48034604j, -0.11909966-0.0208953j ,
         0.05429371+0.05748387j, -0.23184071+0.21897432j,
        -0.61372952+0.08271911j,  0.02050984+0.1521715j ,
         0.05856424+0.04249391j, -0.17138405+0.23619798j], requires_grad=True)

Now we use the same weights and inputs to figure out an equivalent partial fourier series:

In [331]:
embedding_layers, embedding_indices = get_embedding_layers(layers, input_boundaries)
diag_hs, v_daggers = diagonalize_layers(embedding_layers, layer_params, num_qubits)

init_unitaries, unitary_indices = get_unitaries(layers, layer_params, weights, weight_boundaries, 
                                                input_boundaries, num_qubits)

updated_unitaries = update_unitaries(init_unitaries, v_daggers, unitary_indices)

kernel_freqs = compute_freqs(diag_hs, 1)
coeffs = compute_kernel_coeffs(updated_unitaries, diag_hs, kernel_freqs, 2)

state_fourier = get_state_from_coeffs(inputs, coeffs, 2)
state_fourier

array([ 0.21962078+0.05389356j,  0.01336267-0.05445399j,
       -0.0049505 +0.01924282j, -0.07760906-0.0199661j ,
       -0.00712376-0.21751854j, -0.05393275+0.0017663j ,
       -0.0194288 +0.03745217j, -0.15105   -0.07835913j,
        0.08427376-0.48034604j, -0.11909966-0.0208953j ,
        0.05429371+0.05748387j, -0.23184071+0.21897432j,
       -0.61372952+0.08271911j,  0.02050984+0.1521715j ,
        0.05856424+0.04249391j, -0.17138405+0.23619798j])

Now we can compare the two:

In [332]:
np.mean(np.abs(state_qc - state_fourier))

tensor(1.22572157e-16, requires_grad=True)

So the two vectors are the same, meaning that the partial fourier series returned is equivalent to the quantum circuit.

We can also take a look at the different frequencies and coefficients the ansatz gives rise to:

In [333]:
simple_coeffs = simplify_coeffs(coeffs)
print_coeffs(simple_coeffs)

[U(x, θ)|0>]_0 = 

(0.1154884686206302-6.938893903907228e-18j) * exp(-0.5 * i * x_0)  +
(0.1154884686206302+0j) * exp(0.5 * i * x_0)  +
(0.1154884686206302+6.938893903907228e-18j) * exp(0.5 * i * x_1)  +
(0.1154884686206302+1.3877787807814457e-17j) * exp(-0.5 * i * x_1)  +
(0.1154884686206302+0j) * exp(-0.5 * i * x_2)  +
(0.1154884686206302+0j) * exp(0.5 * i * x_2)  +
(0.1154884686206302+6.938893903907228e-18j) * exp(0.5 * i * x_3)  +
(0.1154884686206302+1.3877787807814457e-17j) * exp(-0.5 * i * x_3)  

[U(x, θ)|0>]_1 = 

0j * exp(-0.5 * i * x_0)  +
(6.938893903907228e-18+0j) * exp(0.5 * i * x_0)  +
(-6.938893903907228e-18-6.938893903907228e-18j) * exp(0.5 * i * x_1)  +
0j * exp(-0.5 * i * x_1)  +
0j * exp(-0.5 * i * x_2)  +
0j * exp(0.5 * i * x_2)  +
(0.1154884686206302+6.938893903907228e-18j) * exp(0.5 * i * x_3)  +
(-0.1154884686206302-1.3877787807814457e-17j) * exp(-0.5 * i * x_3)  

[U(x, θ)|0>]_2 = 

0j * exp(-0.5 * i * x_0)  +
0j * exp(0.5 * i * x_0)  +
6.938893903907228e-18j * 

## Conclusions

In this notebook, I presented what is, to the best of my knowledge, the first implementation of a full-fledged demonstration of the the quantum circuit $\rightarrow$ partial fourier series equivalence proven in \[[1]\]. I hope that this helps in the exploration of the properties of variation quantum circuits (and quantum kernel methods) more easily, in particular by allowing for easy access to the coefficients of the partial fourier series of the quantum circuits we create.

[1]: https://arxiv.org/abs/2008.08605

## References

\[1\] Maria Schuld, Ryan Sweke, Johannes Jakob Meyer. The effect of data encoding on the expressive power of variational quantum machine learning models, 2020. [arXiv:2008.08605 [quant-ph]](https://arxiv.org/abs/2008.08605)